# Linear Optimization Assignment 3

## Group Members
- Tanmay Garg CS20BTECH11063
- Aayush Patel CS20BTECH11001
- Ganesh Bombatkar CS20BTECH11016
- Kartik Srinivas ES20BTECH11015

In [21]:
# Suppress the warnings in the notebook for cleaner outputs
# Comment this cell if you dont want to install warnings package
import warnings
warnings.filterwarnings('ignore')

In [22]:
import numpy as np
from numpy import linalg as la
import csv
from scipy.linalg import null_space
np.random.seed(100)

In [23]:
class SimplexAlgorithm():
    def __init__(self, A, b, c, x = None):
        self.A = A
        self.b = b
        self.m, self.n = A.shape
        if x is None:
            # self.x = np.empty([n, 1]) # n,
            pass
        else:
            self.x = x # n,
        self.c = c

        self.eps = 1e-6

        self.make_polytope_non_degenerate()

        assert(self.A.shape == (self.m, self.n))

        assert(self.b.shape == (self.m, ))
        assert(self.b.ndim == 1)

        assert(self.c.shape == (self.n, ))
        assert(self.c.ndim == 1)

        assert(self.x.shape == (self.n, ))

    
    def is_feasiability(self, point):
        return np.all(self.A @ point <= self.b)
        
    def is_vertex(self, point):
        return len(np.where(np.abs(self.A @ point - self.b) <= self.eps)[0]) == self.n
    
    def find_tight_rows(self, point):
        tight_rows = np.where(np.abs(self.A @ point - self.b) <= self.eps)[0]
        return tight_rows
    
    def make_polytope_non_degenerate(self):
        eps_degen = 1e-4
        self.b = self.b + np.geomspace(eps_degen, eps_degen ** self.m, self.m)

    def get_direction(self):
        tight_rows = np.where(np.abs(self.A @ self.x - self.b) <= self.eps)[0]
        # shape of tight_rows: (n, ) since x is a vertex of a non-degenerate polytope)
        assert len(tight_rows) == self.n

        A_tight = self.A[tight_rows]
        # Shape of A_tight: (n, n)
        assert A_tight.shape == (self.n, self.n)

        tight_rows_bool = np.zeros(self.m, dtype=bool) # (m, )
        tight_rows_bool[tight_rows] = True
        untight_rows = np.where(tight_rows_bool == False)[0] # (m - n, )
        assert untight_rows.shape == (self.m - self.n, )

        A_untight = self.A[untight_rows]
        assert A_untight.shape == (self.m - self.n, self.n)

        b_untight = self.b[untight_rows]
        assert b_untight.shape == (self.m - self.n, )

        negative_indices = np.where(la.inv(A_tight.T) @ self.c < 0)[0]
        if len(negative_indices) != 0:
            negative_index = negative_indices[0]
            v = -la.inv(A_tight.T)[negative_index] # (n, )

            if len(np.where(self.A @ v > 0)[0]) == 0:
                # The linear program is unbounded
                return None, 1
            
            dir_epsilon = (b_untight - np.dot(A_untight, self.x)) / (np.dot(A_untight, v)) # (m - n, )
            min_eps = np.min(dir_epsilon[dir_epsilon >= 0])
            return min_eps * v, 0
        else:
            return None, 0
        
    def get_vertex_from_feasible_point(self, alpha=0.1):
        print("Initial feasible point: ", self.x)
        random_direction = np.random.randn(self.n)
        while not self.is_vertex(self.x):
            tight_rows = self.find_tight_rows(self.x)

            A_tight = self.A[tight_rows]
            if len(tight_rows) != 0:
                null_space_vectors = null_space(A_tight, rcond=1e-14)
                direction = null_space_vectors[:, 0]
            else:
                direction = random_direction * 0.01
            
            new_x = self.x
            x = self.x
            alpha_tmp = alpha
            
            num_iter = 0
            dir = 1
            start_x = x
            while True:
                if len(self.find_tight_rows(x)) > len(self.find_tight_rows(self.x)):
                    break

                if num_iter > 10000:
                    # For unbounded polytope, we need to check the direction in the opposite direction as well
                    print("The polytope is unbounded, flipping the direction to find initial vertex")
                    dir *= -1
                    num_iter = 0
                    alpha_tmp = alpha
                    x = start_x
                    continue

                new_x = x + alpha_tmp * direction * dir
                if not self.is_feasiability(new_x):
                    alpha_tmp = alpha_tmp * 0.1
                    continue

                x = new_x
                num_iter += 1
                
            self.x = x
        print("Initial vertex: ", self.x)
        self.x = self.x.round(5)
        print("Initial vertex(After rounding to 5 places): ", self.x)
        return self.x

    def solve(self):
        # Check if the initial point is feasible
        if not self.is_feasiability(self.x):
            print("The initial point is not feasible")
            return
        
        self.x = self.get_vertex_from_feasible_point()

        if not self.is_vertex(self.x):
            print("Point x is not a vertex of the polytope")
            return

        print(f"The initial point is {self.x} with cost {self.c.T @ self.x}")
        
        # Moving from the initial point to a vertex
        iter_count = 0
        while True:
            dir, status = self.get_direction()
            if dir is None:
                if status == 1:
                    print("The linear program is unbounded")
                    return
                elif status == 0:
                    # self.x is the optimal vertex
                    break
            else:
                self.x = self.x + dir
                print(f"At iteration {iter_count + 1}, the current point is {self.x} with cost {self.c.T @ self.x}")
                iter_count += 1

        print(f"The optimal point is {self.x} with cost {self.c.T @ self.x}")

In [24]:
# # bounded cost, unbounded polytope, (1, 11) is a degenerate vertex
# c = np.array([0, -1], dtype=np.float32)
# x = np.array([10, 9], dtype=np.float32)
# b = np.array([10, 10.5, 0, 0, -1], dtype=np.float32)
# A = np.array([[-1, 1], [-0.5, 1], [-1, 0], [0, -1], [-1, 0]], dtype=np.float32)

def read_input_from_csv(file_path):
    with open(file_path, 'r') as csv_file:
        reader = csv.reader(csv_file)
        data = list(reader)

    # Extracting data from CSV
    x = np.array(data[0][:-1], dtype=np.float32)
    c = np.array(data[1][:-1], dtype=np.float32)
    b = np.array(data[2:], dtype=np.float32)[:, -1]
    A = np.array([list(map(float, row[:-1])) for row in data[2:]], dtype=np.float32)

    return x, c, b, A

In [25]:
file_path = './test3.csv'  
x, c, b, A = read_input_from_csv(file_path)

algo = SimplexAlgorithm(A, b, c, x)
algo.solve()

Initial feasible point:  [10. 10.]
Initial vertex:  [9.08477109e-07 5.00000986e+01]
Initial vertex(After rounding to 5 places):  [ 0.     50.0001]
The initial point is [ 0.     50.0001] with cost 50.0001
At iteration 1, the current point is [19.99979973 30.00030027] with cost 109.99949918518067
At iteration 2, the current point is [20.00000009 29.99999973] with cost 110.00000009056637
At iteration 3, the current point is [ 3.00000001e+01 -2.71601692e-07] with cost 120.00000009056637
The optimal point is [ 3.00000001e+01 -2.71601692e-07] with cost 120.00000009056637
